## Q. 기상 실측 데이터를 토대로 태양광 발전량인 amount(2023-10-15) 값을 예측해보시오.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from torch.utils.data import TensorDataset, DataLoader

pd.set_option('display.max_column', 20)
pd.set_option('display.max_row', 100)

data = pd.read_csv('/content/drive/MyDrive/tobigs22/정규세션/5주차/timeserisadvanced/energy_tobigs_question.csv')
data['time'] = pd.to_datetime(data['time'])

# 피처와 타겟 변수 설정
features = ['cloud', 'temp', 'humidity', 'ground_press', 'wind_speed', 'wind_dir', 'rain', 'snow', 'dew_point', 'vis', 'uv_idx', 'azimuth', 'elevation']
target = 'amount'

# 데이터셋 분리 (2023-10-14까지 학습, 2023-10-15 하루 예측)
train_data = data[data['time'] < '2023-10-15'].copy()
test_data = data[(data['time'] >= '2023-10-15') & (data['time'] < '2023-10-16')].copy()

# 학습 데이터에 NaN 값 제거
train_data = train_data.dropna()

# 데이터 스케일링
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

train_data.loc[:, features] = scaler_x.fit_transform(train_data[features])
train_data.loc[:, target] = scaler_y.fit_transform(train_data[[target]])

test_data.loc[:, features] = scaler_x.transform(test_data[features])

### 시퀀스 데이터 생성 함수 작성

In [4]:
# 텐서로 변환해서 텐서데이터셋을 생성하세요. (1~6 번에 내용을 채워보세요.)

# 시퀀스 데이터 생성 함수
# dataX: 시퀀스 데이터를 담은 리스트 (예: [[시퀀스1], [시퀀스2], ...])
# dataY: 각 시퀀스에 대응하는 타겟 값 (예: 발전량)

def build_dataset(time_series, seq_length):
    dataX, dataY = [], []
    for i in range(len(time_series) - seq_length):
        dataX.append(time_series[i:i+seq_length, :-1])
        dataY.append(time_series[i+seq_length, [-1]])
    return np.array(dataX), np.array(dataY)

# 하이퍼파라미터 설정
seq_length = 24
batch_size = 32

# 학습 데이터 생성
# trainX: 24시간의 피처 데이터
# trainY: 그 다음 시간에 대한 발전량 값
trainX, trainY = build_dataset(train_data[features + [target]].values, seq_length)

# 텐서로 변환
trainX_tensor = torch.Tensor(trainX)
trainY_tensor = torch.Tensor(trainY)

# 텐서데이터셋 생성
train_dataset = TensorDataset(trainX_tensor, trainY_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

### 학습한 시계열 모델을 사용해서 예측을 해보세요 (RNN, LSTM, Transformer, Informer 중에서 선택적으로 활용)

In [5]:
# 슬라이딩 윈도우 방식으로 2023-10-15의 24시간 발전량 예측해보기 (Direct Multi-step Forecast Strategy 혹은 Recursive Multi-step Forecast 등 이외의 방법론도 자유롭게 사용 가능)
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)  # 초기 은닉 상태
        out, _ = self.rnn(x, h0)  # RNN을 통해 시퀀스 처리
        out = self.fc(out[:, -1, :])  # 마지막 시퀀스의 출력을 통해 예측
        return out

# 2. 하이퍼파라미터 설정
input_size = len(features)  # 피처 개수
hidden_size = 50  # 은닉층 크기
output_size = 1  # 예측할 타겟의 크기 (발전량)
num_epochs = 20  # 학습 에포크 수

# 3. RNN 모델 생성
model = SimpleRNN(input_size=input_size, hidden_size=hidden_size, output_size=output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 4. 모델 학습
model.train()
for epoch in range(num_epochs):
    for batch_X, batch_Y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 5. 2023-10-15 데이터로 예측
testX = test_data[features].values
testX = np.array([testX])  # 배치 형태로 맞추기
testX_tensor = torch.Tensor(testX)

model.eval()
with torch.no_grad():
    predicted_amounts = model(testX_tensor).cpu().numpy().flatten()

Epoch [1/20], Loss: 0.0026
Epoch [2/20], Loss: 0.0134
Epoch [3/20], Loss: 0.0039
Epoch [4/20], Loss: 0.0091
Epoch [5/20], Loss: 0.0065
Epoch [6/20], Loss: 0.0092
Epoch [7/20], Loss: 0.0068
Epoch [8/20], Loss: 0.0047
Epoch [9/20], Loss: 0.0092
Epoch [10/20], Loss: 0.0175
Epoch [11/20], Loss: 0.0112
Epoch [12/20], Loss: 0.0050
Epoch [13/20], Loss: 0.0126
Epoch [14/20], Loss: 0.0071
Epoch [15/20], Loss: 0.0064
Epoch [16/20], Loss: 0.0090
Epoch [17/20], Loss: 0.0082
Epoch [18/20], Loss: 0.0121
Epoch [19/20], Loss: 0.0042
Epoch [20/20], Loss: 0.0017


### 2023-10-15 발전량 예측값을 predicted_amounts에 저장하고 시각화를 실행해주세요.

In [6]:
# 시각화
fig = go.Figure()

# 실제 발전량 시각화
fig.add_trace(go.Scatter(x=data['time'], y=data['amount'],
                         mode='lines', name='Actual Amount'))

# 예측된 발전량 시각화 (2023-10-15의 24시간 동안의 예측)
fig.add_trace(go.Scatter(x=test_data['time'], y=predicted_amounts,
                         mode='lines', name='Predicted Amount', line=dict(dash='dot', color='red')))

# 레이아웃 설정
fig.update_layout(title='2023-10-15 24시간 발전량 예측',
                  xaxis_title='시간',
                  yaxis_title='발전량 (kWh)')

# 그래프 출력
fig.show()

### 정답을 적어주세요.

In [7]:
# 예측된 24시간 발전량 출력
print(f'2023-10-15 예측 발전량 (24시간): {predicted_amounts} kWh')

2023-10-15 예측 발전량 (24시간): [-0.01927137] kWh


---

### (필수) Informer모델은 transformer 모델의 어떤 부분을 개선하고자 했나요? (차이점을 중심으로 서술)

ProbSparse Self-Attention을 도입. 이는 중요한 쿼리(query)와 키(key)에 대해서만 주의(attention)를 계산하여 희소성(sparsity)을 유도함. 이렇게 하면 복잡도를
𝑂
(
𝐿
log
⁡
𝐿
)
O(LlogL)로 줄여 더 긴 시퀀스에 대해서도 효율적으로 처리할 수 있음

### (필수) 모델링 해석

사실 많은 다양한 모델이 있지만, 시계열 모델중 가장 근본있고 간단해서 빠른 학습이 가능한 작은 사이즈의 rnn을 사용했습니다

In [ ]:
# 모델을 선택했다면 왜 선택했는지 본인만의 근거를 정리해주세요.
# 더 나아가 파라미터 선택의 기준이 있었다면 좋습니다.

### (선택) 데이터 해석

In [ ]:
# 데이터셋을 보고 느낀 생각이나 본인만의 논리 전개 방식을 정리해주세요.
# 전처리를 했다면 해당 전처리를 왜 했는지, 파생변수를 생성했다면 왜 만들었는지 등